### 02 2 Language detection: analysis & results
This notebook dedicated to the analysis of language detection results:
* check the number and variation of languages detected;
* filter errors;
* clean data for visualisations

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [6]:
langs <- read.csv("data/prerevol_sections_lang.csv")

In [7]:
glimpse(langs)

Rows: 223,330
Columns: 3
$ X          <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, …
$ section_id <chr> "ekmteataja18530105.2.1", "ekmteataja18530105.2.2", "ekmtea…
$ lang       <chr> "german", "russian-iso8859_5", "german", "german", "russian…


In [36]:
langs <- langs %>% 
    # add column with the newspaper's name
    mutate(keyid = str_remove_all(section_id, "\\d+|[[:punct:]]")) %>% 
    # clean languages from encoding info as -iso8859_5
    mutate(lang_cln = str_remove_all(lang, "-.*?$"))

langs_groups <- langs %>% 
    group_by(lang_cln) %>% 
    count(sort = T) %>% 
    ungroup() %>% 
    summarise(lang_cln = lang_cln,
              n_total = n, 
              perc = (n_total/sum(n_total))*100)

In [38]:
head(langs_groups, 20)

lang_cln,n_total,perc
<chr>,<int>,<dbl>
russian,126937,56.83831102
german,83578,37.42354363
ukrainian,6257,2.80168361
english,1495,0.66941298
estonian,1299,0.58165047
bulgarian,1078,0.48269377
middle_frisian,886,0.39672234
rumantsch,305,0.13656920
latvian,290,0.12985268


##### Import texts to check language detection restuls

In [19]:
texts <- read.csv("data/prerevol_texts_sections.csv") # very large file of 1GB
str(texts)

'data.frame':	240506 obs. of  3 variables:
 $ X         : int  1 2 3 4 5 6 7 8 9 10 ...
 $ section_id: chr  "ekmteataja18530105.2.1" "ekmteataja18530105.2.2" "ekmteataja18530105.2.3" "ekmteataja18530105.2.4" ...
 $ text      : chr  "Эстляндские губернские ведомости  Издаются еженедsжио по Понсдъльпикамъ Щ,l&gt;ва въ годъ 3 руб. сер., для част"| __truncated__ "I. Отделъ общий.  На основанш 161 §. приложешя къ 648 ст VI продолжен!» 2. тома Свода Закон овъ , распространен"| __truncated__ "I. Allgemeine Abtheilung.  Nach Grundlage des 161. §. der Beilage zum 648. Artikel der VI. Fortsetzung des 2. B"| __truncated__ "II. Местный Отделъ.  II. Oertliche Abtheilung. " ...


In [25]:
langs_joined <- left_join(texts %>% select(-X), 
                          langs %>% select(-X), 
                          by = "section_id") %>% 
                    select(keyid, section_id, text, lang_cln, lang)
str(langs_joined)

'data.frame':	240506 obs. of  5 variables:
 $ keyid     : chr  "ekmteataja" "ekmteataja" "ekmteataja" "ekmteataja" ...
 $ section_id: chr  "ekmteataja18530105.2.1" "ekmteataja18530105.2.2" "ekmteataja18530105.2.3" "ekmteataja18530105.2.4" ...
 $ text      : chr  "Эстляндские губернские ведомости  Издаются еженедsжио по Понсдъльпикамъ Щ,l&gt;ва въ годъ 3 руб. сер., для част"| __truncated__ "I. Отделъ общий.  На основанш 161 §. приложешя къ 648 ст VI продолжен!» 2. тома Свода Закон овъ , распространен"| __truncated__ "I. Allgemeine Abtheilung.  Nach Grundlage des 161. §. der Beilage zum 648. Artikel der VI. Fortsetzung des 2. B"| __truncated__ "II. Местный Отделъ.  II. Oertliche Abtheilung. " ...
 $ lang_cln  : chr  "german" "russian" "german" "german" ...
 $ lang      : chr  "german" "russian-iso8859_5" "german" "german" ...


##### Check
Language of particular interest: 
* the ones that should indeed exist in the local newspapers: Estonian, Latvian;
* the ones that might be language detection tool errors: Ukrainian, English, Bulgarian, Belarus.

In [41]:
languages_local <- c("estonian", "latvian")

langs_joined %>% 
    filter(lang_cln %in% languages_local) %>% 
    group_by(lang_cln) %>% 
    sample_n(10)

keyid      section_id              
1  livzeitung livzeitung18930707.2.8  
2  livzeitung livzeitung18990215.2.4  
3  livzeitung livzeitung19000605.2.12 
4  ekmteataja ekmteataja19070308.2.5.2
5  ekmteataja ekmteataja18841229.2.5.3
6  livzeitung livzeitung19010502.2.13 
7  livzeitung livzeitung18910923.2.16 
8  ekmteataja ekmteataja18880811.2.6.2
9  ekmteataja ekmteataja18930506.2.5.8
10 ekmteataja ekmteataja18890119.2.3.3
11 livzeitung livzeitung18920923.2.8  
12 livzeitung livzeitung18800602.2.2.1
13 livzeitung livzeitung19000310.2.11 
14 livzeitung livzeitung18680605.2.4.1
15 livzeitung livzeitung18930917.2.6  
16 livzeitung livzeitung18931020.2.24 
17 livzeitung livzeitung18950410.2.18 
18 livzeitung livzeitung18720306.2.2.2
19 livzeitung livzeitung18991011.2.25 
20 livzeitung livzeitung18790207.2.2.1
   text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [42]:
languages_error <- c("ukrainian", "english", "bulgarian", "belarus")

langs_joined %>% 
    filter(lang_cln %in% languages_error) %>% 
    group_by(lang_cln) %>% 
    sample_n(10)

keyid      section_id               
1  ekmteataja ekmteataja19131010.2.1.1 
2  livzeitung livzeitung19020814.2.23  
3  ekmteataja ekmteataja18920312.2.7.1 
4  ekmteataja ekmteataja18690604.2.4.4 
5  livzeitung livzeitung18960207.2.5   
6  livzeitung livzeitung19060529.2.5   
7  ekmteataja ekmteataja19050421.2.7.1 
8  livzeitung livzeitung18960923.2.7   
9  ekmteataja ekmteataja18710508.2.4.5 
10 ekmteataja ekmteataja18690410.2.3.3 
11 ekmteataja ekmteataja18871119.2.2   
12 livzeitung livzeitung19090715.2.35  
13 ekmteataja ekmteataja19110804.2.8.1 
14 ekmteataja ekmteataja18731103.2.3.4 
15 ekmteataja ekmteataja19150702.2.5   
16 ekmteataja ekmteataja18910808.2.1   
17 livzeitung livzeitung18970326.2.40  
18 ekmteataja ekmteataja19011213.2.5   
19 livzeitung livzeitung18990628.2.6   
20 ekmteataja ekmteataja18711020.2.3.4 
21 livzeitung livzeitung18640226.2.2   
22 livzeitung livzeitung18600406.2.3   
23 livzeitung livzeitung18531003.2.3   
24 livzeitung livzeitung18700309.2.2   
25 livzeitung livzeitung18550415.2.3   
26 livzeitung livzeitung18580210.2.2   
27 livzeitung livzeitung18700403.2.2   
28 livzeitung livzeitung18551102.2.3   
29 livzeitung livzeitung18720214.2.2   
30 livzeitung livzeitung18550307.2.2   
31 livzeitung livzeitung18621214.2.11.1
32 livzeitung livzeitung18880527.2.10  
33 ekmteataja ekmteataja18860508.2.4   
34 ekmteataja ekmteataja18690823.2.4.5 
35 livzeitung livzeitung19050513.2.8   
36 ekmteataja ekmteataja18890216.2.7.3 
37 ekmteataja ekmteataja18850312.2.2   
38 livzeitung livzeitung19041217.2.24  
39 livzeitung livzeitung19130227.2.18  
40 ekmteataja ekmteataja18901011.2.4   
   text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [43]:
# more pseudo-Ukrainian examples
langs_joined %>% 
    filter(lang_cln == "ukrainian") %>% 
    sample_n(10)

keyid,section_id,text,lang_cln,lang
<chr>,<chr>,<chr>,<chr>,<chr>
ekmteataja,ekmteataja19080911.2.12.1,"(jJLg ГОРОПСНЙХЪ ИЗБИРАТЕЛЕЙ ПО Г. РЕВЕЛЮ 1908 г. Напечатанъ отдЪльнымъ прюжешемъ къ № 37 Зстл. Губ. В""""Ьд. отъ 11-го Сентября 1908 г. г. 1908 r. Типографа Эстляндскаго Губернскаго Правления.",ukrainian,ukrainian-koi8_r
livzeitung,livzeitung18890911.2.3,"НЕОФФИЦИАЛЬНАЯ ЧАСТЬ. C?&gt; Л число Сентябри месяца 1881) г. все письма для мыанаго унраиденш Иоклюае адресовать прем, Смнльтснъ. 91о 1. !pcpkmSei""""i fd}š gada ir wkst pagaflrt sichti'iuim Meksch Siksch»«muischas Waldes par Smilteni suhtami. ПРАВШЕ ТОВАРИЩЕСТВА РИШАГО ЦЕМЕНТНАГО ЗАВОДА И МАШВПЙВИ К. X. Шмидта покорпейгне пробить гг, пайщиков* полой овить mi обыкновенное еобршие 1Vh */Убботу * октябри «что ГОДн н'ь 12 ч о сонь дин, Li, «pnitLHM« в?» Мог KB1I, lift Bupnapuli, пъ ДОМ l&gt; кунощ-гкиго ЩШЦЧТШЦ дла обсуэдсша .- k.дующих.*!) предметен?.: 1} Рааомотрешс и утверждение отчета » баланса за истекипй Tiiuuaiuaтый отчетный годъ. 2) Утперждсше сметным, расходов* н плана дЪйстшй масгулиашаго отчетнаго года. * ' 3) Избран!« одного директора н одного кандидата. 4) Избраше членов?, ронлашиной ком n rein.",ukrainian,ukrainian-koi8_r
ekmteataja,ekmteataja19070628.2.28.1,"- 29 РшьсвШ . . СтарооокодьсвШ СуджаижШ ТВЫСБIЙ . , ФатевсвШ . ЩигровсвlВ 21. Лифляндскан. РижскШ .... Валвсшй . . Вевдевсвгё ФелдивсвlВ ЭзельсЕl3 . 22. Минская. МинсвШ . . . Бобруйшй . . . БорвсовсвШ . . . ИгуыевевШ . . МозырсвШ . . . HoßorpjACEiE . . IIBBCBiS . . . РЪЧПНиШ . . . СлуцкШ . . . .33. Ыошлевская Могилевшй Buiobce'iß . ГоыельсвШ ГорецвШ . КлимовичсеШ МстиславсвИ) oршавскlй . Рогачевсвй Мшвянсеш Чауешв lepasoßcsiß 24. Московски Moi'EOßCüifi Богороде.кш БровввцыВ BepeScEiß . ВолоколамсиШ ДнптровшП Звенигородсвlб Кливскш . Коломевшй МожаВсвШ . НоДОЛЬСЕIЙ . РузевШ CepiiyiOßC.uiö 25 Нижегородская Нижегородец!!! Ардатовшй Арзанассвгё . БадахввнгЕЙ . ВасвльсурсвШ . Горбатовшй . Квягввввсе|В . Лj' KOJiHoucitifi . М аиарьевскН! . Ceueuoßcitifl Cepmcsiß . 137 10! 11 78 137 125 94 10 10 31 41 35 71 12 I 43 5 12 10 6 10 6 6 10 6 71 17 16 10 I 10 12 64 100 21 42 II 13 12 15 13 50 15 1 26. ИоагороОамя БороввзскШ СтарорусскШ ТвхвввсвШ УстюжевсвlЙ ЧерезовецкШ 27 Олонецкая. BHTeropcEiS КарговольсвШ Лодейвопольсвш Оловецкш . . (IовlшецЕlй ИудожишЧ . 28. Оренбургская. ОревбургсвШ . ВерхвеуральсЕlВ OpcKiß . . . Tpoauuiü . . Челябившй . 29. Орловская OBCEiS . БолховскШ . ДмитровскШ ЕлецвШ . . Вврачевсвш Кроыешй . ЛввенсвШ . МалоархавгельсвШ Мцевсшй Ctßciiiö . . Трубчевскш 30. Пензенская Певиеншй . . Городшцеш:кш . КеренсвШ . . Ераснослободсвгё Мокшавтй . НаровчатевШ . Ннж и о л (I S] оисEt i Л Саранлкш . . ЧембарсаШ . 31. Пермская. ПеришЗ . . ВерхотурскlЙ . Еиатеривбургшй ИрбнтсвШ . . БамышливсвЛ! . Ерасвоуфиншй Kyarypcuifi . . Осивсеш ОхавсвШ . . 98 60, 66 16 55 17 26 съ-бгда |уподно--13 1119 32 »I 113 23 58 !17 I I II IH 10 7 7 I 1 6 8 9 11 92 14 1 1 14 6 7 12 6 8 II 22 &gt; 47 II",ukrainian,ukrainian-koi8_r
livzeitung,livzeitung18951108.2.1,"№ 127. Годъ сорвкъ трет 1 й. «805. ■ШФЖНДСКИ ГУБЕРНСКIЯ ВЕДОМОСТИ. Т I ОД.ОЕГ Oligž-M Нагодъ. На выгода. Ka3«tc. Ha Ute P. JL Р. E. Р. К. Р. E. Безъ пересыява н доставай »3 2 1 5С Съ доставкой на доьъ въ город« 4 2 50 1 50 7S Оъ нер зелгкой но очт* ,5 3 2 1 Частная объявлешя ддн наяечатанlя нринилаюмя въ Iн«дяядсвоl Губерневой Тнпограош ежедневно, sa ictHoieiütm ввеервшаъ л праздничных» двай от* 10 чае. утра до 4 чае» ноддо л. Плата ea чаетныя sa строжу въ одан» столбец» . . 6 лов ta строжу в» два столбца . , . 16 , РИГА. Выхедлтл s pasa въ неделю : яо Паиедгдьиккаиъ, Срвдагь я Ляткмяар&gt; Среда, § Ноября.",ukrainian,ukrainian-koi8_r
ekmteataja,ekmteataja18891228.2.4,ЧАСТЬ НЕОФИЦИАЛЬНАЯ. № 50. 28 Декабря 1889 г. № 50.,ukrainian,ukrainian-koi8_r
livzeitung,livzeitung19090701.2.13,"спис о к ъ Имя крестъянскаго двора. на который испрашивается ссуда. Йия дворянскаго шйшл, на которомъ расположенъ втотъ креетьяисий дворъ. Имя заемщика. Юрьево- Во рроскаго У Ъ з д а. Тракси X 31.

In [45]:
langs_samples <- langs_joined %>% 
    filter(lang_cln %in% languages_local | lang_cln %in% languages_error) %>% 
    group_by(lang_cln) %>% 
    sample_n(10)

head(langs_samples %>% ungroup() %>% sample_n(5))

keyid      section_id               
1 livzeitung livzeitung19050530.2.42  
2 ekmteataja ekmteataja19061120.2.18.1
3 livzeitung livzeitung18560924.2.9   
4 livzeitung livzeitung18930514.2.8   
5 livzeitung livzeitung18990924.2.3   
  text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [46]:
# save sampled example for the rmd display
write.csv(file = "data/langs_samples.csv", langs_samples)

##### Some more cleaning for the languages output & joining with metadata

In [47]:
str(langs)

'data.frame':	223330 obs. of  5 variables:
 $ X         : int  1 2 3 4 5 6 7 8 9 10 ...
 $ section_id: chr  "ekmteataja18530105.2.1" "ekmteataja18530105.2.2" "ekmteataja18530105.2.3" "ekmteataja18530105.2.4" ...
 $ lang      : chr  "german" "russian-iso8859_5" "german" "german" ...
 $ keyid     : chr  "ekmteataja" "ekmteataja" "ekmteataja" "ekmteataja" ...
 $ lang_cln  : chr  "german" "russian" "german" "german" ...


In [48]:
metadata <- read.csv("data/prerevol_meta_sections.csv")
str(metadata)

'data.frame':	252993 obs. of  18 variables:
 $ X                                 : int  1 2 3 4 5 6 7 8 9 10 ...
 $ .id                               : int  31 31 31 31 31 31 31 31 31 31 ...
 $ docid                             : chr  "ekmteataja18530105" "ekmteataja18530105" "ekmteataja18530105" "ekmteataja18530105" ...
 $ LogicalSectionFirstPageID         : chr  "ekmteataja18530105.1.1" "ekmteataja18530105.1.1" "ekmteataja18530105.1.1" "ekmteataja18530105.1.1" ...
 $ LogicalSectionID                  : chr  "ekmteataja18530105.2.1" "ekmteataja18530105.2.2" "ekmteataja18530105.2.3" "ekmteataja18530105.2.4" ...
 $ attrs                             : int  1 2 3 4 6 7 8 9 10 11 ...
 $ LogicalSectionTitle               : chr  "Kopfleiste" "I. Отделъ общий." "I. Allgemeine Abtheilung." "II. Местный Отделъ. II. Oertliche Abtheilung." ...
 $ LogicalSectionType                : chr  "TITLE_SECTION+ILLUSTRATION" "ARTICLE" "ARTICLE" "SECTION" ...
 $ LogicalSectionViewURL             : chr  "/cg

In [49]:
meta_langs <- left_join(langs %>% select(-X), 
          metadata %>% 
              select(-X) %>% 
              mutate(section_id = LogicalSectionID),
          by = "section_id")

str(meta_langs)

'data.frame':	223330 obs. of  21 variables:
 $ section_id                        : chr  "ekmteataja18530105.2.1" "ekmteataja18530105.2.2" "ekmteataja18530105.2.3" "ekmteataja18530105.2.4" ...
 $ lang                              : chr  "german" "russian-iso8859_5" "german" "german" ...
 $ keyid                             : chr  "ekmteataja" "ekmteataja" "ekmteataja" "ekmteataja" ...
 $ lang_cln                          : chr  "german" "russian" "german" "german" ...
 $ .id                               : int  31 31 31 31 31 31 31 31 31 31 ...
 $ docid                             : chr  "ekmteataja18530105" "ekmteataja18530105" "ekmteataja18530105" "ekmteataja18530105" ...
 $ LogicalSectionFirstPageID         : chr  "ekmteataja18530105.1.1" "ekmteataja18530105.1.1" "ekmteataja18530105.1.1" "ekmteataja18530105.1.1" ...
 $ LogicalSectionID                  : chr  "ekmteataja18530105.2.1" "ekmteataja18530105.2.2" "ekmteataja18530105.2.3" "ekmteataja18530105.2.4" ...
 $ attrs              

In [51]:
# save meta w/detected languages for viz
write.csv(file = "data/prerevol_meta_langs.csv", meta_langs)